In [1]:
import numpy as np
import cv2
from scipy.spatial import distance


In [2]:
# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return f"{h}:{m:>02}:{s:>05.2f}"

In [3]:
import sys

!{sys.executable} -m pip install git+https://github.com/zzh8829/yolov3-tf2.git@master

  Cloning https://github.com/zzh8829/yolov3-tf2.git (to revision master) to c:\users\dell\appdata\local\temp\pip-req-build-2trixfz8
  Created wheel for yolov3-tf2: filename=yolov3_tf2-0.1-py3-none-any.whl size=9301 sha256=eba2946353989419cde84be620c18289b331d566196a8059809038387389d976
  Stored in directory: C:\Users\dell\AppData\Local\Temp\pip-ephem-wheel-cache-gqiq4_ik\wheels\dc\40\57\f6ce9c0aa58da78f10d29a11476132dbf0a616bb92826be28f
Successfully built yolov3-tf2


  Running command git clone -q https://github.com/zzh8829/yolov3-tf2.git 'C:\Users\dell\AppData\Local\Temp\pip-req-build-2trixfz8'


In [4]:
import tensorflow as tf
import os

ROOT = os.path.join(os.getcwd(),'yolo')

filename_darknet_weights = tf.keras.utils.get_file(
    os.path.join(ROOT,'yolov3.weights'),
    origin='https://pjreddie.com/media/files/yolov3.weights')
TINY = False

filename_convert_script = tf.keras.utils.get_file(
    os.path.join(os.getcwd(),'convert.py'),
    origin='https://raw.githubusercontent.com/zzh8829/yolov3-tf2/master/convert.py')

filename_classes = tf.keras.utils.get_file(
    os.path.join(ROOT,'coco.names'),
    origin='https://raw.githubusercontent.com/zzh8829/yolov3-tf2/master/data/coco.names')
filename_converted_weights = os.path.join(ROOT,'yolov3.tf')

8192/1067 [======================================================================================================================================================================================================================================] - 0s 0us/step


In [5]:
# copy paste the above code in cmd
#python convert.py --weights ./yolo/yolov3.weights --output ./yolo/yolov3.tf

In [6]:
import os
os.remove(filename_convert_script)

In [7]:
import time
from absl import app, flags, logging
from absl.flags import FLAGS
import cv2
import numpy as np
import tensorflow as tf
from yolov3_tf2.models import (YoloV3, YoloV3Tiny)
from yolov3_tf2.dataset import transform_images, load_tfrecord_dataset
from yolov3_tf2.utils import draw_outputs
import sys
from PIL import Image, ImageFile
import requests

# Flags are used to define several options for YOLO.
flags.DEFINE_string('classes', filename_classes, 'path to classes file')
flags.DEFINE_string('weights', filename_converted_weights, 'path to weights file')
flags.DEFINE_boolean('tiny', False, 'yolov3 or yolov3-tiny')
flags.DEFINE_integer('size', 416, 'resize images to')
flags.DEFINE_string('tfrecord', None, 'tfrecord instead of image')
flags.DEFINE_integer('num_classes', 80, 'number of classes in the model')
FLAGS([sys.argv[0]])



['C:\\Users\\dell\\anaconda\\lib\\site-packages\\ipykernel_launcher.py']

In [8]:
if FLAGS.tiny:
    yolo = YoloV3Tiny(classes=FLAGS.num_classes)
else:
    yolo = YoloV3(classes=FLAGS.num_classes)

# Load weights and classes
yolo.load_weights(FLAGS.weights).expect_partial()
print('weights loaded')

class_names = [c.strip() for c in open(FLAGS.classes).readlines()]
print('classes loaded')

weights loaded
classes loaded


In [9]:
video = open("video.mp4")

In [10]:
def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype = "float32")
    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    # return the ordered coordinates
    return rect


def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")
    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    return warped,M




In [11]:
def plot_points(frame,Dict,list_of_close,list_of_connected):

    
    list_of_close =list(list_of_close)
    for key,value  in Dict.items():
        coord1 = value[1]
        coord2 = value[2]
        area = (coord1[0]-coord2[0])*(coord1[1]-coord2[1])
        if area > 8000:
            continue
        if key in list_of_close:
            frame = cv2.rectangle(frame,coord1,coord2,(0,0,255),2)
        else:
            frame = cv2.rectangle(frame,coord1,coord2,(255,0,0),2)
    for connection in list_of_connected:
        key1,key2 = connection.split("-")
        coord1 = Dict[key1][0]
        coord2 = Dict[key2][0]
        frame = cv2.line(frame,coord1,coord2,(0,0,255),1)
        
    return frame
     

In [12]:
def derive_distance(Dict,matrix,scale_factor=1):
    warped_dict = {}
    for key,value in Dict.items():
        warped_dict[key]=return_point(matrix,value[0])
    
    list_of_close=set()
    list_of_connected =set() 
    for key1,value1 in warped_dict.items():
        for key2,value2 in warped_dict.items():
            if key1 == key2:
                continue
            dist = distance.euclidean(value1,value2)*scale_factor
            if dist < 50:
                list_of_connected.add(key1+"-"+key2)
                list_of_close.add(key1)
                list_of_close.add(key2)
        
    return list_of_close,list_of_connected
    
    
        

In [13]:
def create_dict(img, outputs, class_names):
    boxes, objectness, classes, nums = outputs
    boxes, objectness, classes, nums = boxes[0], objectness[0], classes[0], nums[0]
    wh = np.flip(img.shape[0:2])
    Dict={}
    j = 0
    for i in range(nums):
        
        if class_names[int(classes[i])] == 'person':
            x1y1 = tuple((np.array(boxes[i][0:2]) * wh).astype(np.int32))
            x2y2 = tuple((np.array(boxes[i][2:4]) * wh).astype(np.int32))
            (x1,y1) = x1y1
            (x2,y2) = x2y2
            m1 = int((x2-x1)/2)+x1
            m2 = int((y2-y1)/2)+y1
            coord = (m1,m2)
            Dict["person"+str(j)] = [coord,(x1,y1),(x2,y2)]
            j+=1
    
    return Dict
    

In [14]:
def return_point(matrix,p):
    px = (matrix[0][0]*p[0] + matrix[0][1]*p[1] + matrix[0][2]) / ((matrix[2][0]*p[0] + matrix[2][1]*p[1] + matrix[2][2]))
    py = (matrix[1][0]*p[0] + matrix[1][1]*p[1] + matrix[1][2]) / ((matrix[2][0]*p[0] + matrix[2][1]*p[1] + matrix[2][2]))
    p_after = (int(px), int(py))
    return p_after

In [15]:
import cv2
import numpy as np




cap = cv2.VideoCapture('video4.mp4')
width = int(cap.get(3))
height = int(cap.get(4))
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter("output.avi",fourcc, 30,(width,height))




while cap.isOpened():
    ret,frame = cap.read()
    if ret == True:
        frame1 = frame
        width,height,channel = frame.shape
        img = tf.expand_dims(frame, 0)
        img = transform_images(img, FLAGS.size)
        
        boxes, scores, classes, nums = yolo(img)
        
        #cv2.circle(frame,(300,150),5,(0,0,255),-1)
        #cv2.circle(frame,(640,150),5,(0,0,255),-1)
        #cv2.circle(frame,(0,470),5,(0,0,255),-1)
        #cv2.circle(frame,(830,470),5,(0,0,255),-1)
        
        #cv2.circle(frame,(600,300),5,(255,0,0),-1)
        pts1 = np.float32([[300,150],[640,150],[0,470],[830,470]])
        warped ,matrix= four_point_transform(frame,pts1)
        
        Dict = create_dict(frame,(boxes, scores, classes, nums),class_names)
        
        list_of_close,list_of_connected = derive_distance(Dict,matrix,scale_factor=0.8)

        frame = plot_points(frame1,Dict,list_of_close,list_of_connected)
        cv2.imshow("frame",frame)
        #out.write(frame)
        #cv2.imshow("wraped",warped)
        
        
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()